In [1]:
import piheaan
import os

In [2]:
params = piheaan.ParameterPreset.FGb 
context = piheaan.make_context(params) 
piheaan.make_bootstrappable(context) 

# parameter 정보는 https://heaan.it/ 에서 확인 가능

In [3]:
#키생성
sk = piheaan.SecretKey(context) 
key_file_path = "./key" 
os.makedirs(key_file_path, mode=0o775, exist_ok=True) 
log_num_slot = piheaan.get_log_full_slots(context) 
num_slot = 1 << log_num_slot 
sk.save(key_file_path+"/secretkey.bin") 
sk = piheaan.SecretKey(context,key_file_path+"/secretkey.bin") 
key_generator = piheaan.KeyGenerator(context, sk) 
key_generator.gen_common_keys() 
key_generator.save(key_file_path + "/")

In [4]:

#생성된 키 받아오기
sk = piheaan.SecretKey(context) 
key_file_path = "./key" 
log_num_slot = piheaan.get_log_full_slots(context) 
num_slot = 1 << log_num_slot 
sk = piheaan.SecretKey(context,key_file_path+"/secretkey.bin")

keypack = piheaan.KeyPack(context, key_file_path+"/") 
keypack.load_enc_key() 
keypack.load_mult_key() 
eval = piheaan.HomEvaluator(context,keypack) 
dec = piheaan.Decryptor(context) 
enc = piheaan.Encryptor(context)

In [5]:
# 사용 가능한 함수 찾기, argument 찾기
print(dir(eval))
print(help(eval.add))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'add', 'bootstrap', 'bootstrap_extended', 'conjugate', 'i_mult', 'integer_mult', 'kill_imag', 'left_rotate', 'left_rotate_reduce', 'level_down', 'level_down_one', 'min_level_for_bootstrap', 'min_level_for_bootstrap_extended', 'mult', 'mult_without_rescale', 'negate', 'relinearize', 'rescale', 'right_rotate', 'right_rotate_reduce', 'rot_sum', 'square', 'sub', 'tensor']
Help on method add in module piheaan:

add(...) method of piheaan.HomEvaluator instance
    add(*args, **kwargs)
    Overloaded function.
    
    1. add(self: piheaan.HomEvaluator, op1: piheaan.Message, cnst: complex, res: piheaan.Message) -> None
    
    2. add(self: piheaan.HomEvaluator, op1: piheaan.Messa

In [6]:
#1로 가득 찬 암호문 만들기
one_ = [1]*num_slot
one_msg = piheaan.Message(log_num_slot)
for k in range(num_slot):
    one_msg[k] = one_[k]

one_ctxt = piheaan.Ciphertext(context)
enc.encrypt(one_msg, keypack, one_ctxt)

#2로 가득 찬 암호문 만들기
two_ = [2]*num_slot
two_msg = piheaan.Message(log_num_slot)
for k in range(num_slot):
    two_msg[k] = two_[k]

two_ctxt = piheaan.Ciphertext(context)
enc.encrypt(two_msg, keypack, two_ctxt)

In [7]:
#1+2
result_add = piheaan.Ciphertext(context)
eval.add(one_ctxt, two_ctxt, result_add)

add_msg = piheaan.Message(log_num_slot)
dec.decrypt(result_add,sk,add_msg)

print(add_msg[:5])

[(3+0j), (3+0j), (3+0j), (3+0j), (3+0j)]


In [8]:
#1*2
result_mult = piheaan.Ciphertext(context)
eval.mult(one_ctxt, two_ctxt, result_mult)

mult_msg = piheaan.Message(log_num_slot)
dec.decrypt(result_mult,sk,mult_msg)

print(mult_msg[:5])

[(2+0j), (2+0j), (2+0j), (2+0j), (2+0j)]
